In [39]:
from opcua import Client
from opcua import ua
from pprint import pprint
from time import sleep
from datetime import datetime
from collections import namedtuple

In [3]:
client = Client("opc.tcp://10.211.55.26:53530/OPCUA/SimulationServer/")

In [4]:
client.connect()

In [5]:
root = client.get_root_node()

In [6]:
print root

Node(TwoByteNodeId(i=84))


In [7]:
objects = client.get_objects_node()

In [13]:
pprint(objects.get_children())

[Node(FourByteNodeId(i=2253;nsu=http://opcfoundation.org/UA/)),
 Node(StringNodeId(ns=2;s=MyObjectsFolder)),
 Node(StringNodeId(ns=3;s=StaticData)),
 Node(StringNodeId(ns=4;s=NonUaNodeComplianceTest)),
 Node(StringNodeId(ns=5;s=85/0:Simulation)),
 Node(StringNodeId(ns=6;s=MyBigNodeManager))]


In [14]:
sim = client.get_node('ns=5;s=85/0:Simulation')

In [32]:
str(sim.get_children())

'[Node(StringNodeId(ns=5;s=Counter1)), Node(StringNodeId(ns=5;s=Expression1)), Node(StringNodeId(ns=5;s=Random1)), Node(StringNodeId(ns=5;s=Sawtooth1)), Node(StringNodeId(ns=5;s=Sinusoid1)), Node(StringNodeId(ns=5;s=Square1)), Node(StringNodeId(ns=5;s=Triangle1))]'

# What just happened?

We connected to a prosys OPCUA server. This one is used for simulation and will provide repeated patterns that can be identified. Much of this is reapplying examples found at https://github.com/FreeOpcUa. This is not meant to be a demo of OPCUA. Rather, I want to focus on different methods for getting this type of data to down stream datasources where that information can be processed.

In [36]:
exp = client.get_node('ns=5;s=Expression1')

In [37]:
# client.get_node('ns=5;s=Expression11').get_value()

In [26]:
exp.get_value()

3.886289501972594

In [43]:
class PollOPCNode(object):
    """
    Define a tag to be polled
    Provide method to retreive values
    Storage of those values is not in scope for this Class
    """
    
    def __init__(self, node_string, client_obj, clean_string=None):
        self.node_str = node_string
        self.client = client_obj
        self.node = self.client.get_node(self.node_str)
        if clean_string is None:
            self.clean_string = self.node_str
        else:
            self.clean_string = clean_string
        
        # check if this is a valid node
        self._check_for_children()
        self._check_for_value()
        
    def _check_for_children(self):
        """
        Check if a node has children. It is assumped that if a node has children it should not be polled, thus throw an error.
        """
        if len(self.node.get_children()) > 0:
            raise ValueError('This Node is not a leaf node. Children of this node are {}'.format(self.client.get_children()))
    
    def _check_for_value(self):
        """
        It is possible to define a tag that is not valid... check if there is a value associated with that tag.
        """
        self.node.get_value()
        
    def read_value(self, reads=1, _sleep=.1):
        values = []
        Result = namedtuple('Result', ['timestamp', 'fieldvalue', 'fieldname'])
        for count in range(reads):
            if reads > 1:
                sleep(_sleep)
            values.append(Result(datetime.now(), self.node.get_value(), self.clean_string))
        return values

## Testing the polling class above

In [44]:
t = PollOPCNode('ns=5;s=Expression1', client, 'expression')

In [45]:
t.read_value()

[Result(timestamp=datetime.datetime(2017, 12, 28, 17, 19, 37, 53455), fieldvalue=2.7890439238358957, fieldname='expression')]

In [46]:
t.read_value(5)

[Result(timestamp=datetime.datetime(2017, 12, 28, 17, 19, 45, 616900), fieldvalue=4.5813414861761714e-07, fieldname='expression'),
 Result(timestamp=datetime.datetime(2017, 12, 28, 17, 19, 45, 727701), fieldvalue=4.5813414861761714e-07, fieldname='expression'),
 Result(timestamp=datetime.datetime(2017, 12, 28, 17, 19, 45, 838342), fieldvalue=4.5813414861761714e-07, fieldname='expression'),
 Result(timestamp=datetime.datetime(2017, 12, 28, 17, 19, 45, 944946), fieldvalue=4.5813414861761714e-07, fieldname='expression'),
 Result(timestamp=datetime.datetime(2017, 12, 28, 17, 19, 46, 55069), fieldvalue=-0.015822859966911462, fieldname='expression')]

In [47]:
t.read_value(5, .5)

[Result(timestamp=datetime.datetime(2017, 12, 28, 17, 20, 18, 917557), fieldvalue=0.024429749891246688, fieldname='expression'),
 Result(timestamp=datetime.datetime(2017, 12, 28, 17, 20, 19, 424133), fieldvalue=0.11371077187895073, fieldname='expression'),
 Result(timestamp=datetime.datetime(2017, 12, 28, 17, 20, 19, 934624), fieldvalue=0.11371077187895073, fieldname='expression'),
 Result(timestamp=datetime.datetime(2017, 12, 28, 17, 20, 20, 440849), fieldvalue=0.26794945593034947, fieldname='expression'),
 Result(timestamp=datetime.datetime(2017, 12, 28, 17, 20, 20, 951108), fieldvalue=0.26794945593034947, fieldname='expression')]